# Анализ базы данных сервиса для чтения книг по подписке

## Цели, задачи и описание данных проекта

**Цель:** На основе данных сформулировать рекомендации для приложения.

**Задачи:**

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры);
- Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Таблица books** - данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

**Таблица authors** - данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

**Таблица publishers** - данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

**Таблица ratings** - данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

**Таблица reviews** - данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

## Проверка и подготовка данных к анализу

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
#Вывод 5 строк по таблице с книгами
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
#Вывод 5 строк по таблице с авторами
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
#Вывод 5 строк по таблице с издательствами
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
#Вывод 5 строк по таблице с рейтингами
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
#Вывод 5 строк по таблице с обзорами
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Количество книг

Посчитать, сколько книг вышло после 1 января 2000 года.

In [7]:
#Вывод 5 строк по таблице с книгами
query = '''
        SELECT COUNT(*)
        FROM books
        WHERE publication_date > '2000-01-01'
        '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Всего вышло 819 книг после 1 января 2000 года.

## Количество обзоров и средняя оценка по каждой книге

In [8]:
query = '''
        SELECT 
        b.book_id, b.title, rt.avg_rating, rv.review_count
        FROM books b
        LEFT JOIN 
            (SELECT book_id, AVG(rating) AS avg_rating 
             FROM ratings 
             GROUP BY book_id) 
             rt ON b.book_id=rt.book_id
        LEFT JOIN 
            (SELECT book_id, COUNT(*) AS review_count 
             FROM reviews 
             GROUP BY book_id) 
             rv ON b.book_id=rv.book_id
        ORDER BY avg_rating DESC, review_count DESC
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,avg_rating,review_count
0,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
1,17,A Dirty Job (Grim Reaper #1),5.00,4.0
2,553,School's Out—Forever (Maximum Ride #2),5.00,3.0
3,444,Moneyball: The Art of Winning an Unfair Game,5.00,3.0
4,913,The Woman in Black,5.00,2.0
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,202,Drowning Ruth,2.00,3.0
997,371,Junky,2.00,2.0
998,316,His Excellency: George Washington,2.00,2.0


Выведена таблица с номером и названием книги, средний рейтинг и количество обзоров по каждой из книг.
По предварительным данным видно, что не у всех книг есть обзоры.
Также количество обзоров примерно в рамках от 2х до 4х. У книг и с высоким и с низким рейтингом примерно одинаковое количество обзоров.

## Издательство, которое выпустило наибольшее число книг

Убрать книги, которые толще 50 страниц, чтобы исключить из анализа брошюры

In [9]:
query = '''
        SELECT b.publisher_id,p.publisher, COUNT(*) AS books_count
        FROM books AS b
        LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
        WHERE num_pages > 50
        GROUP BY b.publisher_id, p.publisher
        ORDER BY books_count DESC
        LIMIT 1
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,books_count
0,212,Penguin Books,42


Издательство "Penguin Books" выпустило наибольшое количество книг - 42. В данное количество не входят брошюры.

## Автор с самой высокой средней оценкой книг

Необходимо учитывать только книги с 50 и более оценками

In [10]:
query = '''
        SELECT a.author, AVG(ratings) AS avg_rating
        FROM (SELECT book_id, AVG(rating) ratings
              FROM ratings
              GROUP BY book_id
              HAVING COUNT(*) >= 50) rat
        JOIN (SELECT book_id, author_id 
              FROM books) b ON rat.book_id=b.book_id
        JOIN (SELECT author_id, author 
              FROM authors) a ON b.author_id=a.author_id
        GROUP BY a.author
        ORDER BY avg_rating DESC
        LIMIT 1
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Джоан Роулинг является автором с самой высокой средней оценкой книг - 4.28.

## Среднее количество обзоров от пользователей

Учитывать пользователей, которые поставили больше 48 оценок

In [11]:
query = '''
        WITH rating AS (
            SELECT ratings.username, COUNT(rating) AS ratings_quantity
            FROM ratings
            GROUP BY ratings.username
            HAVING COUNT(rating) > 48),
             review AS (
            SELECT reviews.username,
            COUNT(text) AS reviews_count
            FROM reviews
            GROUP BY reviews.username)
        SELECT AVG(reviews_count)
        FROM rating
        LEFT JOIN review ON review.username = rating.username
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составило 24.

## Вывод

Информация, полученная в ходе анализа:
- В базе данных находятся данные о 1000 книгах.
- После 1 января 2000 года вышло 819 книг.
- По предварительным данным видно, что не у всех книг есть обзоры. Также количество обзоров примерно в рамках от 2х до 4х. У книг и с высоким и с низким рейтингом примерно одинаковое количество обзоров.
- Джоан Роулинг является автором с самой высокой средней оценкой книг - 4.28.
- Издательство "Penguin Books" выпустило наибольшое количество книг - 42. В данное количество не входят брошюры.
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составило 24.